In [3]:
%load_ext autoreload

In [10]:
import os
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.multiprocessing as mp

In [4]:
%autoreload 2

from syncbn import SyncBatchNorm

In [5]:
sbn = SyncBatchNorm(100)
bn = nn.BatchNorm1d(100)

In [6]:
x = torch.rand(16, 100, 256)

In [8]:
# sbn(x)

In [20]:
def run(rank, size, input):
    """ Distributed function to be implemented later. """
    out = sbn(input)

def init_process(rank, size, input, fn, backend='gloo'):
    """ Initialize the distributed environment. """
    os.environ['MASTER_ADDR'] = '127.0.0.1'
    os.environ['MASTER_PORT'] = '29500'
    
    dist.init_process_group(backend, rank=rank, world_size=size)
    fn(rank, size, input)

In [21]:
size = 2
x = torch.rand(size, 16, 100, 256)

processes = []
mp.set_start_method("spawn")
for rank in range(size):
    p = mp.Process(target=init_process, args=(rank, size, x[rank], run))
    p.start()
    processes.append(p)

for p in processes:
    p.join()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/kvtamogashev/miniforge3/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/kvtamogashev/miniforge3/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'init_process' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/kvtamogashev/miniforge3/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/kvtamogashev/miniforge3/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'init_process' on <module '__main__' (built-in)>


In [19]:
x[0].shape

torch.Size([16, 100, 256])

In [25]:
torch.randn(10, 10, 10).mean(dim=(0, 1), keepdims=True)

tensor([[[ 0.0784,  0.0996, -0.0383, -0.0372, -0.0514,  0.0836, -0.1197,
           0.1208,  0.0244, -0.1221]]])